In [1]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import L1L2,l2, l1
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.utils.layer_utils import layer_from_config
from keras.layers import deserialize as layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras import backend as K
K.set_image_dim_ordering('th')


In [3]:
%matplotlib inline

import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

In [4]:

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))


In [5]:
from skimage.transform import rescale, resize, downscale_local_mean
from utils import preprocess_bw
from sklearn import svm
import skimage
import cv2
import os
import numpy as np

In [6]:
def load_data(path):
    X = []
    y = []
    for label in os.listdir(path):
        if os.path.isdir(os.path.join(path,label)):
            for filename in os.listdir(os.path.join(path,label)):
                if not filename.startswith("."):
                    img = cv2.imread(os.path.join(path,label,filename), 0)
                    #preprocessed = preprocess_bw(img)
                    output = skimage.transform.resize(img, (20,20), mode="reflect",preserve_range=True)
                    X.append(output)
                    y.append(int(label))
    X = np.asarray(X)
    y = np.asarray(y)
    return X,y

In [56]:
X, y = load_data("training2")

In [57]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=42)

In [58]:
batch_size=64
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [40]:
X_train.shape

(5760, 1, 20, 20)

In [41]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
def norm_input(x): return (x-mean_px)/std_px

In [42]:
model = Sequential([
    Lambda(norm_input, input_shape=(1,20,20)),
    Convolution2D(32,3,3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(),
    BatchNormalization(axis=1),
    Convolution2D(64,3,3, activation='relu'),
    BatchNormalization(axis=1),
    Convolution2D(64,3,3, activation='relu'),
    MaxPooling2D(),
    Flatten(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(10, activation='softmax')
    ])

model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  """
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_5 (Lambda)            (None, 1, 20, 20)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 18, 18)        320       
_________________________________________________________________
batch_normalization_21 (Batc (None, 32, 18, 18)        128       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 16, 16)        9248      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 32, 8, 8)          0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 32, 8, 8)          128       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 64, 6, 6)          18496     
__________

In [43]:
def fit_model(model):
    model.fit_generator(batches, batches.n, nb_epoch=1, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    model.optimizer.lr=0.1
    model.fit_generator(batches, batches.n, nb_epoch=4, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    model.optimizer.lr=0.01
    model.fit_generator(batches, batches.n, nb_epoch=12, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    model.optimizer.lr=0.001
    model.fit_generator(batches, batches.n, nb_epoch=18, verbose=0,
                        validation_data=test_batches, nb_val_samples=test_batches.n)
    return model

In [44]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [45]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='model/weights.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1)

In [46]:
model.fit_generator(batches, batches.n, nb_epoch=10, verbose=1,
                        validation_data=test_batches, nb_val_samples=test_batches.n, callbacks=[checkpointer])

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., 5760, verbose=1, validation_steps=1440, epochs=10, validation_data=<keras.pre..., callbacks=[<keras.ca...)`
  


Epoch 1/10
5760/5760 [==============================] - 145s - loss: 0.0405 - acc: 0.9875 - val_loss: 0.0173 - val_acc: 0.9965
Epoch 2/10
5760/5760 [==============================] - 146s - loss: 0.0043 - acc: 0.9987 - val_loss: 4.6812e-05 - val_acc: 1.0000
Epoch 3/10
5760/5760 [==============================] - 147s - loss: 0.0028 - acc: 0.9993 - val_loss: 0.0046 - val_acc: 0.9979
Epoch 4/10
5760/5760 [==============================] - 147s - loss: 0.0022 - acc: 0.9995 - val_loss: 0.0047 - val_acc: 0.9986
Epoch 5/10
5760/5760 [==============================] - 147s - loss: 0.0015 - acc: 0.9996 - val_loss: 0.0214 - val_acc: 0.9951
Epoch 6/10
5760/5760 [==============================] - 147s - loss: 0.0016 - acc: 0.9996 - val_loss: 0.0020 - val_acc: 0.9986
Epoch 7/10
5760/5760 [==============================] - 147s - loss: 0.0010 - acc: 0.9997 - val_loss: 7.7623e-06 - val_acc: 1.0000
Epoch 8/10
5760/5760 [==============================] - 147s - loss: 7.8947e-04 - acc: 0.9998 - val_los

In [59]:
y_result = model.predict(X_test, batch_size=256)

In [60]:
np.argmax(y_test, axis=-1), np.max(y_test, axis=-1)

(array([8, 2, 7, ..., 0, 8, 8]), array([ 1.,  1.,  1., ...,  1.,  1.,  1.]))